<a href="https://colab.research.google.com/github/heesukjang/W209_DataViz_Summer2023/blob/main/Unit_Assignments/Week_09_Assignment_ChoroplethMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## W209 Data Visualization, Summer 2023
**Week 9 Assignment<br>
Heesuk Jang**

### All tracks<br>

This weeks assignment is to plot a county-level choropleth using the "unemployment" dataset available here: https://cdn.jsdelivr.net/npm/vega-datasets@1.29.0/data/.

You can load this dataset in Observable by importing the JS package, see example here.

In Python, you can load this dataset directly from

**from vega_datasets import data<br>
source = data.unemployment.url**

**For Tableau**, you can download the TSV file from the link.

In [1]:
!pip install nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from vega_datasets import data
source = data.unemployment.url
source

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

In [2]:
import altair as alt
import pandas as pd
import numpy as np
import networkx as nx
import nx_altair as nxa
from itertools import combinations
import json
from collections import defaultdict
import community.community_louvain as community_louvain

# Gdrive Mount using gmail account
from google.colab import drive
drive.mount('/content/gdrive')

pd.options.display.float_format = '{:.0f}'.format
pd.__version__

Mounted at /content/gdrive


'1.5.3'

## Read JSON file from Google Drive (Gmail Account)

In [3]:
with open('/content/gdrive/MyDrive/W209_DataViz/duto_guerra_tweets.json') as f:
  file = f.read()
  data = json.loads(file)

# two hashtags appear together in a tweet
print(f"len(data) = {len(data)}\nhashtags: count_edges = {len(data[181]['entities']['hashtags'])}, {data[181]['entities']['hashtags']}\n")
data[181]

len(data) = 5872
hashtags: count_edges = 3, [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]



{'retweeted': False,
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'entities': {'user_mentions': [{'name': 'RStudio',
    'screen_name': 'rstudio',
    'indices': ['18', '26'],
    'id_str': '235261861',
    'id': '235261861'}],
  'urls': [{'url': 'https://t.co/sN6CjOYIhH',
    'expanded_url': 'http://johnguerra.co/viz/influentials/RStudioConf2018/',
    'display_url': 'johnguerra.co/viz/influentia…',
    'indices': ['114', '137']}],
  'symbols': [],
  'media': [{'expanded_url': 'https://twitter.com/duto_guerra/status/959841361070567425/photo/1',
    'indices': ['160', '183'],
    'url': 'https://t.co/FzHtxIYfVP',
    'media_url': 'http://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'id_str': '959840986485657600',
    'id': '959840986485657600',
    'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'sizes': {'medium': {'w': '600', 'h': '558', 'resize': 'fit'},
     'thumb': {'w': '150', 'h': '150', 're

In [4]:
lt = data[181]['entities']['hashtags']
print("A tweet that contains three hashtags (=>'rstudioconf','DataViz','DataScience'):\n\n",lt)

A tweet that contains three hashtags (=>'rstudioconf','DataViz','DataScience'):

 [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]


In [5]:
# [{'text': 'dataviz', 'indices': ['109', '117']}]
# [{'text': 'UIST2020', 'indices': ['13', '22']}, {'text': 'AR', 'indices': ['101', '104']}]
# [{'text': 'dataViz', 'indices': ['119', '127']}]
# [{'text': 'deckgl', 'indices': ['82', '89']}, {'text': 'dataviz', 'indices': ['169', '177']}, {'text': 'WebDev', 'indices': ['178', '185']}]
# [{'text': 'vegaLite', 'indices': ['107', '116']}, {'text': 'WebDev', 'indices': ['136', '143']}]

In [6]:
for i in range(len(data)-5800):
  if len(data[i]['entities']['hashtags']) > 0:
    print(i, data[i]['entities']['hashtags'])

4 [{'text': 'dataviz', 'indices': ['40', '48']}]
17 [{'text': 'dataviz', 'indices': ['225', '233']}]
18 [{'text': 'dataviz', 'indices': ['267', '275']}]
20 [{'text': 'dataviz', 'indices': ['230', '238']}]
21 [{'text': 'dataviz', 'indices': ['96', '104']}]
23 [{'text': 'd3js', 'indices': ['60', '65']}]
26 [{'text': 'CSS', 'indices': ['92', '96']}]
36 [{'text': 'clearviewai', 'indices': ['51', '63']}]
44 [{'text': 'dataviz', 'indices': ['109', '117']}]
47 [{'text': 'UIST2020', 'indices': ['13', '22']}, {'text': 'AR', 'indices': ['101', '104']}]
50 [{'text': 'dataViz', 'indices': ['119', '127']}]
52 [{'text': 'dataviz', 'indices': ['121', '129']}]
56 [{'text': 'ieeevis', 'indices': ['33', '41']}]
57 [{'text': 'dataviz', 'indices': ['79', '87']}]
58 [{'text': 'eurovis', 'indices': ['48', '56']}, {'text': 'dataviz', 'indices': ['272', '280']}]
63 [{'text': 'eurovis', 'indices': ['31', '39']}]
66 [{'text': 'dataviz', 'indices': ['77', '85']}]
70 [{'text': 'EuroVis2020', 'indices': ['74', '86

## Extract **Nodes** (= Hashtags) and **Edges** between Two Hashtags at a Time

**itertools.combinations:**
itertools.combinations() provides us with **all the possible tuples** a sequence or set of numbers or letters used in the iterator and the elements are assumed to be unique on the basis of their positions which are distinct for all elements.

In [7]:
# https://stackoverflow.com/questions/49274565/create-2-tuples-from-a-list

hashtag_edges = []

# for i in range(len(data)-5800):
for i in range(len(data)):
  # extract hashtag from each tweet
  hashtag = data[i]['entities']['hashtags']

  # if the hashtag field is not empty, pull the name of hashtag from a tweet, using the key, 'text'
  if len(hashtag) > 0:
    hashtags = [ht['text'].lower() for ht in hashtag]

    # if there are more than one hashtag in a tweet,
    if len(hashtag) > 1:
      hashtag_edge = combinations(hashtags, 2)   # create a list of tuples of two hashtags, representing an edge
      hashtag_edges.extend(sorted(e) for e in hashtag_edge)  # create a master list of sorted edges

print('hashtag_edges: ',hashtag_edges)
print([tuple(l) for l in hashtag_edges])

hashtag_edges:  [['ar', 'uist2020'], ['dataviz', 'eurovis'], ['a11y', 'dataviz'], ['tactileart', 'tactiledrawing'], ['nicar18', 'rstats'], ['datavis', 'infodesign'], ['feedly', 'infovis'], ['colorpicker', 'dataviz'], ['d3js', 'dataviz'], ['dataviz', 'rstudioconf'], ['datascience', 'rstudioconf'], ['datascience', 'dataviz'], ['blind', 'braille'], ['atia18', 'braille'], ['atia18', 'blind'], ['d3js', 'pokeviz'], ['100daysofdataviz', 'dataviz'], ['d3js', 'datascience'], ['dataviz', 'rusia2018'], ['rusia2018', 'visualizacion'], ['dataviz', 'visualizacion'], ['d3js', 'dataviz'], ['cfp', 'ieeevis'], ['dataviz', 'treemap'], ['d3js', 'treemap'], ['d3js', 'dataviz'], ['reactjs', 'regl'], ['regl', 'webdev'], ['reactjs', 'webdev'], ['d3js', 'webdev'], ['webdev', 'webgl'], ['dataviz', 'deckgl'], ['deckgl', 'webdev'], ['dataviz', 'webdev'], ['vegalite', 'webdev'], ['vegalite', 'webdev'], ['proudteacher', 'webdev'], ['semiotic', 'starwars'], ['covidー19', 'dataviz'], ['ai', 'machinelearning'], ['ai', 

## **Count Edges** Where Two Hashtags Appear Together in a Tweet

In [8]:
# Assign clusters using "community_louvain.best_partition(G)"
G = nx.Graph()
G.add_edges_from([tuple(l) for l in hashtag_edges])
partition = community_louvain.best_partition(G)
print('partition: ', partition)

# Count edges
edge_dict = defaultdict(int)
for edge in hashtag_edges:
  edge_dict[tuple(edge)] += 1
# print(edge_dict)

# Create a data table to create a bubble chart in Tableau
data_table =[]
for k, v in edge_dict.items():
  table_dict = {'source_node': k[0],
                'target_node': k[1],
                'count_edge': v,
                'cluster_source': partition[k[0]],
                'cluster_target': partition[k[1]]}

  data_table.append(table_dict)

# Save the output data into a JSON and then import this file to Tableau for the plot
with open('hashtag_nodes_and_edges_v3.json', 'w') as f:
  json.dump(data_table, f)

data_table[:10]

partition:  {'ar': 0, 'uist2020': 0, 'dataviz': 27, 'eurovis': 27, 'a11y': 8, 'tactileart': 3, 'tactiledrawing': 3, 'nicar18': 4, 'rstats': 4, 'datavis': 6, 'infodesign': 6, 'feedly': 6, 'infovis': 6, 'colorpicker': 27, 'd3js': 7, 'rstudioconf': 8, 'datascience': 8, 'blind': 9, 'braille': 9, 'atia18': 9, 'pokeviz': 7, '100daysofdataviz': 27, 'rusia2018': 10, 'visualizacion': 10, 'cfp': 4, 'ieeevis': 4, 'treemap': 7, 'reactjs': 12, 'regl': 12, 'webdev': 12, 'webgl': 12, 'deckgl': 12, 'vegalite': 16, 'proudteacher': 12, 'semiotic': 14, 'starwars': 14, 'covidー19': 48, 'ai': 16, 'machinelearning': 18, 'metoo': 27, 'netviz': 27, 'bogotá': 18, 'revealjs': 7, 'hcil': 8, 'projectlincoln': 27, 'javascript': 12, 'statistics': 18, 'visualization': 16, 'immersive2017': 4, 'testoftime': 6, 'vizsec': 4, 'deeplearning': 18, 'tensorflow': 18, '2q17': 19, 'btw17': 19, 'meteorjs': 12, 'assets2017': 16, 'chi2018': 16, 'elasticsearch': 27, 'matplotlib': 20, 'pandas': 20, 'ggplot2': 20, 'nodejs': 12, 'mong

[{'source_node': 'ar',
  'target_node': 'uist2020',
  'count_edge': 1,
  'cluster_source': 0,
  'cluster_target': 0},
 {'source_node': 'dataviz',
  'target_node': 'eurovis',
  'count_edge': 1,
  'cluster_source': 27,
  'cluster_target': 27},
 {'source_node': 'a11y',
  'target_node': 'dataviz',
  'count_edge': 4,
  'cluster_source': 8,
  'cluster_target': 27},
 {'source_node': 'tactileart',
  'target_node': 'tactiledrawing',
  'count_edge': 1,
  'cluster_source': 3,
  'cluster_target': 3},
 {'source_node': 'nicar18',
  'target_node': 'rstats',
  'count_edge': 1,
  'cluster_source': 4,
  'cluster_target': 4},
 {'source_node': 'datavis',
  'target_node': 'infodesign',
  'count_edge': 1,
  'cluster_source': 6,
  'cluster_target': 6},
 {'source_node': 'feedly',
  'target_node': 'infovis',
  'count_edge': 17,
  'cluster_source': 6,
  'cluster_target': 6},
 {'source_node': 'colorpicker',
  'target_node': 'dataviz',
  'count_edge': 1,
  'cluster_source': 27,
  'cluster_target': 27},
 {'source_

## Testing Code

In [9]:
# import networkx as nx
# import community.community_louvain as community_louvain

# # Example list of tuples representing edges in a network
# edges = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 1), (3, 6), (6, 7), (7, 3)]

# # Create a graph from the edges
# graph = nx.Graph()
# graph.add_edges_from(edges)

# # Run the Louvain algorithm
# partition = community_louvain.best_partition(graph)
# print('partition: ', partition)
# # Retrieve the clusters
# clusters = {}
# for node, cluster_id in partition.items():
#     print(f'node = {node}, cluster_id = {cluster_id}')
#     if cluster_id not in clusters:
#         clusters[cluster_id] = []
#     clusters[cluster_id].append(node)

# # Print the clusters
# for cluster_id, nodes in clusters.items():
#     print(f"Cluster {cluster_id}: {nodes}")